In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds

**Chargement du jeu de données MNIST depuis TensorFlow Datasets.**

In [4]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist', split=['train', 'test'], as_supervised=True, with_info=True
)

Fonction de prétraitement des données :


*   Convertit l'image en float32 et normalise les pixels (valeurs entre 0.0 et 1.0).
*   Transforme l'image 2D (28x28) en vecteur 1D (784).



In [5]:
def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return tf.reshape(image, [-1]), label

**Prétraitement des données et regroupement en mini-batchs de 64 exemples.**

In [6]:
ds_train = ds_train.map(preprocess).batch(64)
ds_test = ds_test.map(preprocess).batch(64)

Initialisation des poids et biais du modèle :

*   W1, b1 : pour la couche cachée.
*   W2, b2 : pour la couche de sortie.



In [7]:
W1 = tf.Variable(tf.random.normal([784, 128], stddev=0.1))
b1 = tf.Variable(tf.zeros([128]))
W2 = tf.Variable(tf.random.normal([128, 10], stddev=0.1))
b2 = tf.Variable(tf.zeros([10]))

Définition du modèle :


*   Couche cachée avec ReLU.
*   Couche de sortie produisant les logits.



In [8]:
def model(x):
    x = tf.nn.relu(tf.matmul(x, W1) + b1)
    return tf.matmul(x, W2) + b2

**La fonction de perte intègre le softmax en interne grâce à `from_logits=True`**

In [9]:
loss_fn = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.optimizers.Adam(0.001)

**Phase d'entrainement**

In [10]:
for epoch in range(5):
    for x_batch, y_batch in ds_train:
        #  Démarrage de l'enregistrement automatique des opérations pour le calcul des gradients
        with tf.GradientTape() as tape:
            logits = model(x_batch)
            loss = loss_fn(y_batch, logits)
        # Calcul des gradients de la perte par rapport aux paramètres du modèle
        grads = tape.gradient(loss, [W1, b1, W2, b2])
        # Application des gradients pour mettre à jour les paramètres (descente de gradient)
        optimizer.apply_gradients(zip(grads, [W1, b1, W2, b2]))
    print(f"Epoch {epoch+1} terminé")

Epoch 1 terminé
Epoch 2 terminé
Epoch 3 terminé
Epoch 4 terminé
Epoch 5 terminé


**Evaluation du modele**

In [11]:
correct = 0
total = 0
for x_batch, y_batch in ds_test:
    logits = model(x_batch)

    # Pour chaque echantillon du batch, on selectionne l’indice de la classe avec la plus grande valeur de logit
    preds = tf.argmax(logits, axis=1, output_type=tf.int32)

    y_batch = tf.cast(y_batch, tf.int32)

    correct += tf.reduce_sum(tf.cast(preds == y_batch, tf.int32))
    total += x_batch.shape[0]
print("Accuracy :", correct.numpy() / total)

Accuracy : 0.9735
